# Mnist classification demo

In [1]:
import grpc 
import numpy as np
import hydro_serving_grpc as hs
from grpc import ssl_channel_credentials
import keras
from keras.datasets import mnist
from keras import backend as K
import os

/Users/nikitagurov/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/nikitagurov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Read data sample

In [5]:
Xtest = np.load('activity_recognition/data/SHL5hip_300_X_test.npy')[:,0]
val = np.array(Xtest)[1].reshape(1,300,3,3).astype(np.float32)
val = val.flatten()

(2700,)

## Setup channel

In [6]:

channel = grpc.secure_channel("dev.k8s.hydrosphere.io", credentials=ssl_channel_credentials())
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="activity_recognition", signature_name="infer")

## Create tensors

In [7]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=300),hs.TensorShapeProto.Dim(size=3),hs.TensorShapeProto.Dim(size=3)])
tensor = hs.TensorProto(dtype=hs.DT_DOUBLE, tensor_shape=tensor_shape, double_val=val)

## Send request to model servable

In [8]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"x": tensor})
result = stub.Predict(request)
print('Output values:')
print(result)
# print('Ground truth label:')
# print(y_test[0])

Output values:
outputs {
  key: "y"
  value {
    dtype: DT_DOUBLE
    tensor_shape {
      dim {
        size: 5
      }
    }
    double_val: 0.003309800988063216
    double_val: 0.9832870960235596
    double_val: 0.010390128009021282
    double_val: 0.00010289066995028406
    double_val: 0.0029099872335791588
  }
}

